In [2]:
import pandas as pd
import numpy as np
import math
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook

# 기상환경데이터_TXT를 CSV파일로 저장

In [25]:
# 폴더 안에 있는 데이터 모두 대입해서 데이터 프레임 만들기
from os import listdir
from os.path import isfile, join

files = [f for f in listdir('../data/weather/') if isfile(join('../data/weather/', f))]
files
        
# 파일 경로명 리스트 생성하기
file_path_list = []
for file in files :
    path = '../data/weather/'
    file_path_list.append(path+file)

file_path_list

['../data/weather/TV_WEATHER_DATA_1MIN_201805.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201806.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201807.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201808.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201809.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201810.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201811.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201812.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201901.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201902.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201903.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201904.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201905.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201906.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201907.txt',
 '../data/weather/TV_WEATHER_DATA_1MIN_201908.txt']

In [35]:
# 파일 경로 넣으면 데이터 프레임 뽑아주는 함수 생성하기
def make_wind_df(path) :
    data = pd.read_csv(path)
    
    data_srs = data.iloc[:,0]
    data_srs[0].replace("`","").split("|")
    
    STN = []
    TM = []
    DRTN = []
    DRTN2 = []
    SPD = []

    for line in data_srs :
        line_list = line.replace("`","").split("|")
        STN.append(line_list[0])
        TM.append(line_list[1])
        DRTN.append(line_list[2])
        DRTN2.append(line_list[3])
        SPD.append(line_list[4])
    
    date_list = []
    hour_list = []
    min_list = []
    for i in TM :
        date_list.append(str(i)[:8])
    for i in TM :
        hour_list.append(str(i)[8:10])
    for i in TM :
        min_list.append(str(i)[10:12])


    df = pd.DataFrame({"station":STN,
                                "date":date_list,
                                "hour":hour_list,
                                "min":min_list,
                                "wind_drtn":DRTN,
                                "wind_drtn_2":DRTN2,
                                "wind_speed":SPD})
    
    file_name = path[-8:-4]
#     print("%s일자 데이터프레임 미리보기 :\n" %file_name, df.head(3))
    
    df.set_index("station",inplace=True) # 나중에 불러올때 unname 행 안가지고 오기위해 설정
    df.to_csv("../data/weather/%s.csv" %file_name, encoding="euc-kr")


def weather_date(raw) :
    year_list = []
    month_list = []
    day_list = []
    for i in raw.date :
        year_list.append(str(i)[:4])
    for i in raw.date :
        month_list.append(str(i)[4:6])
    for i in raw.date :
        day_list.append(str(i)[6:])

    # 리스트 세개를 합쳐 새로운 dataframe 생성
    split_df = pd.DataFrame({"year":year_list,
                            "month":month_list,
                            "day":day_list})

    # 필요한 열만 가져와서 병합
    raw = raw.loc[:,['station','wind_drtn','wind_drtn_2','wind_speed']]
    result = pd.concat([split_df,raw], axis=1)
    

    
    return result


def read_file(file_path) :
    raw = pd.read_csv(file_path, encoding = 'euc-kr')
    return weather_date(raw)

In [36]:
# 확인하기
path_sample = file_path_list[0]
make_wind_df(path_sample)

1805일자 데이터프레임 미리보기 :
   station      date hour min wind_drtn wind_drtn_2 wind_speed
0  [기] 서울  20180505   02  06     296.2         서북서         .7
1  [기] 서울  20180505   01  42     196.4         남남서        2.7
2  [기] 서울  20180505   01  43     187.2           남        1.8


# Write CSV File

In [37]:
for path in file_path_list :
    make_wind_df(path)

1805일자 데이터프레임 미리보기 :
   station      date hour min wind_drtn wind_drtn_2 wind_speed
0  [기] 서울  20180505   02  06     296.2         서북서         .7
1  [기] 서울  20180505   01  42     196.4         남남서        2.7
2  [기] 서울  20180505   01  43     187.2           남        1.8
1806일자 데이터프레임 미리보기 :
   station      date hour min           wind_drtn wind_drtn_2  \
0  [기] 도봉  20180601   00  30  358.70001220703125           북   
1  [기] 도봉  20180601   00  33   322.1000061035156          북서   
2  [기] 도봉  20180601   00  35  346.29998779296875         북북서   

          wind_speed  
0   .699999988079071  
1  .4000000059604645  
2   .699999988079071  
1807일자 데이터프레임 미리보기 :
   station      date hour min wind_drtn wind_drtn_2 wind_speed
0  [기] 서울  20180718   19  03       276           서          2
1  [기] 서울  20180718   19  04       301         서북서          2
2  [기] 서울  20180718   19  05       289         서북서          2
1808일자 데이터프레임 미리보기 :
    station      date hour min   wind_drtn wind_drtn_2 wind_speed
0 

# Manual

In [3]:
#데이터 불러오기
data = pd.read_csv("../data/weather/TV_WEATHER_DATA_1MIN_201805.txt")

#데이터 모양 확인
data.head()

,`SCAPTION`|`SECTIME`|`S00A`|`S00A_S`|`S01A`|`S02A`|`S03M`|`S04A`|`S05A`|`S06M`|`S07M`
0,`[기] 서울`|`20180505020600`|`296.2`|`서북서`|`.7`|`...
1,`[기] 서울`|`20180505014200`|`196.4`|`남남서`|`2.7`|...
2,`[기] 서울`|`20180505014300`|`187.2`|`남`|`1.8`|`1...
3,`[기] 서울`|`20180505014400`|`194.9`|`남남서`|`1.6`|...
4,`[기] 서울`|`20180505014500`|`223.1`|`남서`|`2.2`|`...


In [4]:
# 컬럼 리스트 뽑아내고 길이 확인
col_name = list(data.columns)[0].replace("`","").split("|")
print("column name list :",col_name,"\ncolumn name list length :",len(col_name))

column name list : ['SCAPTION', 'SECTIME', 'S00A', 'S00A_S', 'S01A', 'S02A', 'S03M', 'S04A', 'S05A', 'S06M', 'S07M'] 
column name list length : 11


In [5]:
# 데이터 프레임 시리즈화
data_srs = data.iloc[:,0]

# 시리즈 첫줄 데이터 분할 후 컬럼 길이와 동일한지 확인
data_srs[0].replace("`","").split("|")
len(data_srs[0].replace("`","").split("|")) # 동일하므로 해당 내용으로 함수 작성 

11

In [6]:
# 필요한 데이터 : SCAPTION(지점명) - 0번째, SECTIVE(년월일시분) - 1번째, SOOA(풍향) - 2번째,
# SOOA_S(풍향) - 3번째, SO1A(풍속) - 4번째
STN = []
TM = []
DRTN = []
DRTN2 = []
SPD = []

for line in data_srs :
    line_list = line.replace("`","").split("|")
    STN.append(line_list[0])
    TM.append(line_list[1])
    DRTN.append(line_list[2])
    DRTN2.append(line_list[3])
    SPD.append(line_list[4])

In [7]:
# 데이터 길이 일치여부 확인
print("staion length :",len(STN))
print("time stemp length :",len(TM))
print("wind direction length :",len(DRTN))
print("wind direction 2 length :",len(DRTN2))
print("wind speed length :",len(SPD))

staion length : 2107799
time stemp length : 2107799
wind direction length : 2107799
wind direction 2 length : 2107799
wind speed length : 2107799


In [8]:
TM[0]

'20180505020600'

In [9]:
# 새로운 dataframe 으로 작성
date_list = []
hour_list = []
min_list = []
for i in TM :
    date_list.append(str(i)[:8])
for i in TM :
    hour_list.append(str(i)[8:10])
for i in TM :
    min_list.append(str(i)[10:12])


weather_1805 = pd.DataFrame({"station":STN,
                            "date":date_list,
                            "hour":hour_list,
                            "min":min_list,
                            "wind_drtn":DRTN,
                            "wind_drtn_2":DRTN2,
                            "wind_speed":SPD})
weather_1805.head()

,station,date,hour,min,wind_drtn,wind_drtn_2,wind_speed
0,[기] 서울,20180505,02,06,296.2,서북서,.7
1,[기] 서울,20180505,01,42,196.4,남남서,2.7
2,[기] 서울,20180505,01,43,187.2,남,1.8
3,[기] 서울,20180505,01,44,194.9,남남서,1.6
4,[기] 서울,20180505,01,45,223.1,남서,2.2


In [17]:
# index 일반 정수로 설정할 경우 다시 read_csv할때 unname으로 읽어옴 > index 미리 station으로 설정하고 저장하기
weather_1805.set_index("station",inplace=True)

In [18]:
weather_1805.to_csv("../data/weather/sample2.csv",encoding="euc-kr")

In [19]:
# 잘 저장됐는지 확인
data_sample = pd.read_csv("../data/weather/sample2.csv",encoding="euc-kr")
data_sample.head()

,station,date,hour,min,wind_drtn,wind_drtn_2,wind_speed
0,[기] 서울,20180505,2,6,296.2,서북서,0.7
1,[기] 서울,20180505,1,42,196.4,남남서,2.7
2,[기] 서울,20180505,1,43,187.2,남,1.8
3,[기] 서울,20180505,1,44,194.9,남남서,1.6
4,[기] 서울,20180505,1,45,223.1,남서,2.2


# 기상_rawfile date 구분

In [ ]:
files = [f for f in listdir('../data/기상_rawfile_csv/') if isfile(join('../data/기상_rawfile_csv/', f))]

file_path_list = []
for file in files :
    path = '../data/기상_rawfile_csv/'
    file_path_list.append(path+file)

In [ ]:
for i in tqdm_notebook(file_path_list):
    name = i.split('/')[-1][:4]
    exec("wind_%s = read_file('%s')" %(name,i)) 

name = []
for i in file_path_list:
    name.append(i.split('/')[-1][:4])

for i in tqdm_notebook(name):
    exec("%s.to_csv('../data/기상_rawfile_csv/기상_file_csv/%s.csv', encoding='euc-kr')" %('wind_'+i,i))

# 결측치 제거하여 저장

In [ ]:
files = [f for f in listdir('/Users/mac/Desktop/bigcam/weather_rawfile_csv/') if isfile(join('/Users/mac/Desktop/bigcam/weather_rawfile_csv/', f))]
        
# 파일 경로명 리스트 생성하기
file_path_list = []
for file in files :
    path = '/Users/mac/Desktop/bigcam/weather_rawfile_csv/'
    file_path_list.append(path+file)

In [ ]:
#결측값 함수
def deleteMissingValue_df(path):
    df = pd.read_csv(path, encoding = 'euc-kr')

    df_drop_row = df.dropna()
    
    file_name = path[-8:-4]
    print("%s일자 데이터프레임 미리보기 :\n" %file_name, df_drop_row.head(3))
    
    df_drop_row.set_index("station",inplace=True)
    df_drop_row.to_csv("/Users/mac/Desktop/bigcam/weather_noMissingV_csv/%s.csv" %file_name, encoding="euc-kr")

In [ ]:
for path in file_path_list :
    deleteMissingValue_df(path)

1809일자 데이터프레임 미리보기 :
   station      date  hour  min  wind_drtn wind_drtn_2  wind_speed
0  [기] 서울  20180901     0    0      324.5          북서         0.2
1  [기] 강남  20180901     0    0      299.5         서북서         0.9
2  [기] 서초  20180901     0    0      147.5         남남동         0.9
1808일자 데이터프레임 미리보기 :
    station      date  hour  min   wind_drtn wind_drtn_2  wind_speed
0  [기] 북악산  20180826     1   35  321.100006          북서         0.0
1  [기] 북악산  20180826     1   36  317.200012          북서         0.0
2  [기] 북악산  20180826     1   37  312.799988          북서         0.0
1908일자 데이터프레임 미리보기 :
   station      date  hour  min   wind_drtn wind_drtn_2  wind_speed
0  [서] 강동  20190801    24    0    0.000000           북         0.0
1  [서] 은평  20190801    24    0    0.000000           북         0.0
2  [서] 중구  20190801    24    0  281.799988         서북서         0.8
1905일자 데이터프레임 미리보기 :
   station      date  hour  min   wind_drtn wind_drtn_2  wind_speed
0  [서] 강동  20190501    24    0    0.00000

# 이상치 제거

In [ ]:
files = [f for f in listdir('/Users/mac/Desktop/bigcam/weather_noMissingV_csv/') if isfile(join('/Users/mac/Desktop/bigcam/weather_noMissingV_csv/', f))]
files

['1809.csv',
 '1808.csv',
 '1908.csv',
 '1905.csv',
 '1904.csv',
 '1906.csv',
 '1907.csv',
 '1903.csv',
 '1902.csv',
 '1901.csv',
 '1806.csv',
 '1812.csv',
 '1807.csv',
 '1811.csv',
 '1805.csv',
 '1810.csv']

In [ ]:
file_path_list = []
for file in files :
    path = '/Users/mac/Desktop/bigcam/weather_noMissingV_csv/'
    file_path_list.append(path+file)

In [ ]:
def outliers_iqr(data):
    return np.where(data < 0)

def deleteOutliers_df(path):
    df = pd.read_csv(path, encoding = 'euc-kr')
    
    '''
    plt.figure(figsize=(7, 7))
    boxplot = df.boxplot(column=['wind_speed'])
    plt.yticks(np.arange(0, 30, step=5))
    plt.show()
    '''
    
    for i in outliers_iqr(df.wind_speed)[0] :
        print(df[i:i+1].wind_speed)
        df.drop(i)
        
    file_name = path[-8:-4]
    print("%s일자 데이터프레임 미리보기 :\n" %file_name, df.head(10))
    
    df.set_index("station",inplace=True)
    df.to_csv("/Users/mac/Desktop/bigcam/weather_noOutliers_csv/%s.csv" %file_name, encoding="euc-kr")

In [ ]:
for path in file_path_list :
    deleteOutliers_df(path)

996340   -3264.0
Name: wind_speed, dtype: float64
1762827   -0.1
Name: wind_speed, dtype: float64
1809일자 데이터프레임 미리보기 :
    station      date  hour  min   wind_drtn wind_drtn_2  wind_speed
0   [기] 서울  20180901     0    0  324.500000          북서         0.2
1   [기] 강남  20180901     0    0  299.500000         서북서         0.9
2   [기] 서초  20180901     0    0  147.500000         남남동         0.9
3   [기] 강동  20180901     0    0   11.300000         북북동         0.0
4   [기] 송파  20180901     0    0  288.000000         서북서         0.8
5   [기] 강서  20180901     0    0   34.000000          북동         0.9
6   [기] 양천  20180901     0    0  317.799988          북서         0.6
7   [기] 도봉  20180901     0    0  316.200012          북서         0.5
8   [기] 노원  20180901     0    0   87.699997           동         0.4
9  [기] 동대문  20180901     0    0   96.099998           동         0.7
1808일자 데이터프레임 미리보기 :
    station      date  hour  min   wind_drtn wind_drtn_2  wind_speed
0  [기] 북악산  20180826     1   35  321.10000

1902일자 데이터프레임 미리보기 :
    station      date  hour  min   wind_drtn wind_drtn_2  wind_speed
0  [기] 영등포  20190201    23   59  147.500000         남남동         0.9
1   [기] 관악  20190201    23   59  196.300003         남남서         1.6
2   [기] 남현  20190201    23   59  259.399994           서         0.0
3   [기] 강북  20190201    23   59  335.899994         북북서         0.2
4   [기] 구로  20190201    23   59  211.399994         남남서         1.1
5   [기] 성동  20190201    23   59  233.300003          남서         1.8
6   [기] 중구  20190201    23   59  246.000000         서남서         0.7
7   [기] 한강  20190201    23   59  239.699997         서남서         2.6
8   [기] 금천  20190201    23   59  188.899994           남         1.4
9   [기] 은평  20190201    23   59    0.000000           북         0.0
65535   -0.1
Name: wind_speed, dtype: float64
65984   -0.1
Name: wind_speed, dtype: float64
1041650   -0.1
Name: wind_speed, dtype: float64
1041899   -0.1
Name: wind_speed, dtype: float64
1901일자 데이터프레임 미리보기 :
    station      date